ErrorException with message 'syntax error, unexpected '<', expecting end of file'

In [5]:
//include "connection.php";

//$connection = SshTunneledConnectionFactory::create_from_environment();
//print_r($connection->query("SELECT * FROM nanohub.jos_faq"));

In [6]:
class SshConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

class MysqlConfig {
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

class SshTunneledDb2Connection 
{
    public $ssh_config;
    public $mysql_config;
    
    public function __construct(SshConfig $ssh_config, MysqlConfig $mysql_config)
    {
        $this->ssh_config = $ssh_config;
        $this->mysql_config = $mysql_config;
    }
    
    public function query(string $query)
    {
        $connection = ssh2_connect($this->ssh_config->hostname, $this->ssh_config->port);
        if($connection === FALSE){
            throw new RuntimeException("Failed to connect with SSH server.");
        }

        if(!ssh2_auth_password($connection, $this->ssh_config->username, $this->ssh_config->password)){
              throw new RuntimeException("Failed to authenticate to SSH server");
        }
        
        $ssh_query ='ssh -L 3307:' . $this->ssh_config->hostname.':' .$this->mysql_config->port.'; 
        echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.$this->mysql_config->username.' -h '.$this->mysql_config->hostname.' --password='.$this->mysql_config->password;
    
        $response = ssh2_exec($connection, $ssh_query);
        
        $error = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
        $error_response = stream_get_contents($error);
        //if(!is_null($error_response))
        //    throw new RuntimeException(sprintf("Error: %s", $error_response));

        stream_set_blocking($response, true);
        stream_set_blocking($error, true);

        return stream_get_contents($response);
    }
}

class SshTunneledConnectionFactory
{
    static public function create_from_environment()
    {
        $ssh_config = new SshConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $mysql_config = new MysqlConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return new SshTunneledDb2Connection($ssh_config, $mysql_config);
    }
}

In [7]:
$connection = SshTunneledConnectionFactory::create_from_environment();
print_r($connection->query("SELECT * FROM nanohub.jos_faq"));

id	title	alias	params	fulltxt	created	created_by	modified	modified_by	checked_out	checked_out_time	state	access	hits	version	section	category	helpful	nothelpful
\n</p>	2011-03-31 15:10:54	62	2011-03-31 15:10:54	62	0	0000-00-00 00:00:00	1	0	3	1	1	0	0	0
\n\t</ul>	2011-03-31 17:21:07	69	2011-03-31 17:21:07	69	0	0000-00-00 00:00:00	1	0	2	1	2	0	0	0
3	Why can\\'t I log in?	login	show_date=2\nallow_comments=0\nclose_comments=year\nfeeds_enabled=1\nfeed_entries=partial	<p>Have you registered? You must register in order to log in. If you have registered and you still cannot log in then check and double-check your username and password. Usually this is the problem; if not, contact <a href=\\"/support/ticket/new/\\" title=\\"Report problem form\\">support</a>.\n</p>	2010-03-25 06:26:40	62	0000-00-00 00:00:00	0	0	0000-00-00 00:00:00	1	0	1	1	2	0	0	0
\n</ul>	2010-03-25 06:26:40	62	0000-00-00 00:00:00	0	0	0000-00-00 00:00:00	1	0	0	1	2	0	0	0
\n</p>	2010-03-25 06:26:40	62	0000-00-00 00:00:00	0	0	0000-0

true

In [ ]:
// $ssh['username'] = getenv('tunnel_ssh_username');
// $ssh['password'] = getenv('tunnel_ssh_password');
// $ssh['hostname'] = getenv('tunnel_ssh_host');
// $ssh['port'] = getenv('tunnel_ssh_port');

// $mysql['username'] = getenv('db_user');
// $mysql['password'] = getenv('db_password');
// $mysql['hostname'] = getenv('db_host');
// $mysql['port'] = getenv('tunnel_remote_bind_port');

In [ ]:
$query = 'SELECT * FROM nanohub.jos_faq';

$ssh_query ='ssh -L 3307:' . getenv('tunnel_ssh_host').':' . getenv('tunnel_remote_bind_port').'; 
echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.getenv('db_user').' -h '.getenv('db_host').' --password='.getenv('db_password');

$response = ssh2_exec($connection, $ssh_query);
$error_response = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
print_r(stream_get_contents($error_response));

stream_set_blocking($response, true);
stream_set_blocking($error_response, true);

$results = stream_get_contents($response);
if (is_null($results ))
    echo "Empty response/result";

else 
    print_r($results);

ssh2_disconnect ( $connection );